In [117]:
# import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from meteostat import Stations, Daily, Hourly
import requests
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [118]:
auth_url = "https://www.strava.com/oauth/token"
activites_url = "https://www.strava.com/api/v3/athlete/activities"

payload = {
    'client_id': "114186",
    'client_secret': '535f4e9fce8216e83d23ecf9644b9b3c00f05ba5',
    'refresh_token': '3f8ab023c1f397d9fa98ed15ca94977267deb54c',
    'grant_type': "refresh_token",
    'f': 'json'
}

# First API call to request access token
# Access tokens expire, so this saves you manual work of regenerating access tokens manually
print("Requesting Token...\n")
res = requests.post(auth_url, data=payload, verify=False)
access_token = res.json()['access_token']
print("Access Token = {}\n".format(access_token))

# Make API request to retrieve the last 400 activities
header = {'Authorization': 'Bearer ' + access_token}
param1 = {'per_page': 200, 'page': 1}
param2 = {'per_page': 200, 'page': 2}
param3 = {'per_page': 200, 'page': 3}
my_dataset1 = requests.get(activites_url, headers=header, params=param1).json()
my_dataset2 = requests.get(activites_url, headers=header, params=param2).json()
my_dataset3 = requests.get(activites_url, headers=header, params=param3).json()

print(my_dataset1[0]["name"])
print(my_dataset1[0]["map"]["summary_polyline"])

print(my_dataset2[0]["name"])
print(my_dataset2[0]["map"]["summary_polyline"])

print(my_dataset3[0]["name"])
print(my_dataset3[0]["map"]["summary_polyline"])

Requesting Token...

Access Token = bc3e2b2968e083267c6a69116aea736268546f1c

TAPER TIME
yvniE```mQ]`HYdA}@rAcCfAeHn@qKmAuBoBsEyBgOaKaAfBmAfAoFv@qJAwEdBoCk@yAkBcAk@iC?sAZuKdH_B^cGAg@lWwBrQG`DkLIoHaB_d@iAaIXaOa@gHdA]S`@sCOo@t@Wv@oAxEqEdHgChBeBjB}DReCe@yCbCuDw@oCKwE?aA|@_F_@q@wBw@c@mA}Au@ZqAtAeCj@eEt@IbLtGfBtAFZMJ}@m@]{@^kAx@aAr@sDdAw@bHVnBy@xBiF?cAqAmB`DXfAe@lAsBCyCpAo@dAmAlBm@lBqFpAWpA{@hFxBJo@}@}AScBVe@~@a@zA?bA`CvBfChAB`@oAp@yKqAiAqC|FqAp@[s@NmDfCoCh@wApBaC^wAtF{@vCkCtAkEt@yN`Hm@`EcA~@m@^e@AwEXw@pDmEzAW|@gBj@]~AgE`Bc@LYCoEYwCmAaGh@y@zAw@bDI~@}EFwDRiA{@mEuHyDi@s@Gq@w@Pg@c@oCTgEMuEl@}Ug@yB{@qCiEuBmA{LYF{GbAiUHst@C{LMmHcd@SsAWR_Cdf@HXkKUqFVsACy@LSjBJJtLYv@oAI_@fCtBNPzBIzL\vBBnPUzZRpQ]fAK~Ca@nB{AbAu@pBW`Oc@TsCOu@mAqS]wHo@_B@s@pA}Zy@IQJkMNe@tEKxTt@`@`@JfBtCLr@t@vBDdD|@dPf@t@z@ClF`RX|BtAtBvDzB~@pWb@`Gi@pCNpD[~@NnGqDn@wC@cA|@_@Pe@[{Bh@oBBeBZi@WcBAaEv@{AhAT?bETjBzAdFVnCMnAa@`@z@x@Bh@_BzFkAlAgBnF[b@o@\b@fBDfDS|@E|Ck@~CYV_DBoBtAKn@pAlF`@jGt@RdAzBb@hBTdD\\UpLv@nFdA`CtBbCtKlGj@sHgAyFX{@n@UiAcFAu

In [119]:
# Flatten nested JSON response into a Dataframe
activities1 = pd.json_normalize(my_dataset1)
activities2 = pd.json_normalize(my_dataset2)
activities3 = pd.json_normalize(my_dataset3)

# Combine into one dataframe
activities = pd.concat([activities1, activities2, activities3])

print("Dataframe shape: ", activities.shape, "\n")
print("Dataframe columns: \n", activities.columns)
activities[['start_latlng','location_city', 'location_state', 'average_temp', 'athlete.resource_state']]


Dataframe shape:  (600, 55) 

Dataframe columns: 
 Index(['resource_state', 'name', 'distance', 'moving_time', 'elapsed_time',
       'total_elevation_gain', 'type', 'sport_type', 'workout_type', 'id',
       'start_date', 'start_date_local', 'timezone', 'utc_offset',
       'location_city', 'location_state', 'location_country',
       'achievement_count', 'kudos_count', 'comment_count', 'athlete_count',
       'photo_count', 'trainer', 'commute', 'manual', 'private', 'visibility',
       'flagged', 'gear_id', 'start_latlng', 'end_latlng', 'average_speed',
       'max_speed', 'average_cadence', 'has_heartrate', 'average_heartrate',
       'max_heartrate', 'heartrate_opt_out', 'display_hide_heartrate_option',
       'elev_high', 'elev_low', 'upload_id', 'upload_id_str', 'external_id',
       'from_accepted_tag', 'pr_count', 'total_photo_count', 'has_kudoed',
       'suffer_score', 'athlete.id', 'athlete.resource_state', 'map.id',
       'map.summary_polyline', 'map.resource_state', 'ave

,start_latlng,location_city,location_state,average_temp,athlete.resource_state
0,"[33.175002029165626, -96.668865038082]",None,None,NaN,1
1,[],None,None,NaN,1
2,"[33.1751259136945, -96.6688531357795]",None,None,NaN,1
3,"[32.86181917414069, -96.93371423520148]",None,None,NaN,1
4,"[33.17498375661671, -96.66915505193174]",None,None,NaN,1
...,...,...,...,...,...
195,"[30.27222507633269, -97.63271743431687]",None,None,NaN,1
196,"[30.272474605590105, -97.63263487257063]",None,None,NaN,1
197,"[29.96328371576965, -90.05960329435766]",None,None,NaN,1
198,"[29.96969662606716, -90.08095753379166]",None,None,NaN,1


In [120]:
# Create DataFrame with specific columns of interest
cols = ['start_date_local', 'sport_type', 'start_latlng', 'distance', 'moving_time',  'total_elevation_gain', 'type', 'average_speed', 
        'max_speed','average_cadence', 'average_heartrate', 'max_heartrate', 'elev_high']

my_activities = activities[cols]

# Break Start Date into Time + Date columns
my_activities['start_date_local'] = pd.to_datetime(activities['start_date_local'])
my_activities['start_time'] = my_activities['start_date_local'].dt.time
my_activities['start_date_local'] = my_activities['start_date_local'].dt.date
my_activities['start_lat'] = round(my_activities['start_latlng'].str[0], 2)
my_activities['start_lng'] = round(my_activities['start_latlng'].str[1], 2)

my_activities.head()

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

,start_date_local,sport_type,start_latlng,distance,moving_time,total_elevation_gain,type,average_speed,max_speed,average_cadence,average_heartrate,max_heartrate,elev_high,start_time,start_lat,start_lng
0,2023-10-22,Run,"[33.175002029165626, -96.668865038082]",35424.9,9692,231.7,Run,3.655,5.278,81.9,159.9,183.0,214.6,09:33:50,33.18,-96.67
1,2023-10-21,Swim,[],225.0,414,0.0,Swim,0.543,3.571,NaN,87.5,109.0,0.0,22:18:21,NaN,NaN
2,2023-10-21,Run,"[33.1751259136945, -96.6688531357795]",9676.9,2659,65.7,Run,3.639,4.494,81.0,152.2,177.0,220.0,13:08:26,33.18,-96.67
3,2023-10-21,Walk,"[32.86181917414069, -96.93371423520148]",3308.1,3192,3.7,Walk,1.036,1.508,48.5,86.6,102.0,132.6,08:44:20,32.86,-96.93
4,2023-10-20,Run,"[33.17498375661671, -96.66915505193174]",16113.3,4482,114.5,Run,3.595,4.635,80.6,156.5,175.0,207.3,12:24:38,33.17,-96.67


In [121]:
# Data Prep for Model Training

# Keep only sport_type == 'Run'
run_activities = my_activities.loc[my_activities['sport_type'] == 'Run']
run_activities = run_activities.drop(columns=['sport_type', 'start_latlng'])
print("Run Activities Shape:",run_activities.shape)

# Find date range and location list for querying external weather data
min_date, max_date = run_activities['start_date_local'].min(), run_activities['start_date_local'].max()
run_locations = run_activities.loc[:,['start_lat', 'start_lng']].drop_duplicates().values

print("Min Activity Date:",min_date,"\nMax Activitiy Date:", max_date)
run_activities.head()

Run Activities Shape: (490, 14)
Min Activity Date: 2022-02-19 
Max Activitiy Date: 2023-10-22


,start_date_local,distance,moving_time,total_elevation_gain,type,average_speed,max_speed,average_cadence,average_heartrate,max_heartrate,elev_high,start_time,start_lat,start_lng
0,2023-10-22,35424.9,9692,231.7,Run,3.655,5.278,81.9,159.9,183.0,214.6,09:33:50,33.18,-96.67
2,2023-10-21,9676.9,2659,65.7,Run,3.639,4.494,81.0,152.2,177.0,220.0,13:08:26,33.18,-96.67
4,2023-10-20,16113.3,4482,114.5,Run,3.595,4.635,80.6,156.5,175.0,207.3,12:24:38,33.17,-96.67
6,2023-10-19,12903.1,3522,88.3,Run,3.664,4.930,81.3,153.5,173.0,209.4,11:12:57,33.17,-96.67
7,2023-10-18,8072.6,2296,71.5,Run,3.516,4.221,82.4,148.1,165.0,207.2,11:58:41,33.18,-96.67


In [122]:
print(run_locations)

[[  33.18  -96.67]
 [  33.17  -96.67]
 [  30.25  -97.75]
 [  30.26  -97.75]
 [  29.44  -98.48]
 [  39.66 -104.94]
 [  39.77 -105.04]
 [  30.24  -97.72]
 [  45.59 -122.33]
 [  45.97 -123.42]
 [  45.61 -122.8 ]
 [  45.31 -121.84]
 [  29.61  -98.59]
 [  29.44  -98.56]
 [  29.44  -98.55]
 [  29.43  -98.48]
 [  33.86  -96.66]
 [  29.4   -98.49]
 [  41.95  -87.65]
 [ -33.02  -71.56]
 [ -22.91  -68.2 ]
 [ -33.43  -70.64]
 [  39.74 -104.94]
 [  39.74 -104.96]
 [  40.65 -111.49]
 [  40.65 -111.5 ]
 [  30.28  -97.75]
 [  30.29  -97.75]
 [  30.24  -97.75]
 [  29.44  -98.49]
 [  27.81  -97.08]
 [  27.7   -97.29]
 [  27.71  -97.32]
 [  27.79  -97.39]
 [  37.76 -121.44]
 [  37.32 -119.56]
 [  29.45  -98.49]
 [  29.61  -98.51]
 [  30.27  -97.76]
 [  30.3   -97.74]
 [  29.73  -95.69]
 [  29.64  -96.52]
 [  29.53  -97.18]
 [  29.5   -97.44]
 [    nan     nan]
 [  29.41  -98.5 ]
 [  34.07   -4.98]
 [  33.6    -7.61]
 [  52.37    4.87]
 [  52.37    4.88]
 [  69.64   18.94]
 [  29.41  -98.49]
 [  30.26  -

In [123]:
# Get unique list of weather stations from run locations
weather_stations = pd.DataFrame()
run_loc_dict = {}
for location in run_locations:
    stations = Stations()
    nearby_station = stations.nearby(location[0], location[1]).fetch(1)
    run_loc_dict[str(location)] = nearby_station.index[0]
    weather_stations = pd.concat([weather_stations, nearby_station])

weather_stations = weather_stations.drop_duplicates(subset='name')
print("weather_stations shape:",weather_stations.shape)
print(run_loc_dict)
weather_stations.head()


weather_stations shape: (59, 16)
{'[ 33.18 -96.67]': 'KTKI0', '[ 33.17 -96.67]': 'KTKI0', '[ 30.25 -97.75]': '72254', '[ 30.26 -97.75]': '72254', '[ 29.44 -98.48]': '72253', '[  39.66 -104.94]': 'KAPA0', '[  39.77 -105.04]': '72469', '[ 30.24 -97.72]': '74745', '[  45.59 -122.33]': 'KTTD0', '[  45.97 -123.42]': '72791', '[  45.61 -122.8 ]': 'KVUO0', '[  45.31 -121.84]': 'KCZK0', '[ 29.61 -98.59]': '72253', '[ 29.44 -98.56]': 'KSKF0', '[ 29.44 -98.55]': 'KSKF0', '[ 29.43 -98.48]': 'KSSF0', '[ 33.86 -96.66]': 'KGYI0', '[ 29.4  -98.49]': 'KSSF0', '[ 41.95 -87.65]': 'KCGX0', '[-33.02 -71.56]': 'SCRD0', '[-22.91 -68.2 ]': '85432', '[-33.43 -70.64]': '85577', '[  39.74 -104.94]': '72469', '[  39.74 -104.96]': '72469', '[  40.65 -111.49]': 'K36U0', '[  40.65 -111.5 ]': 'K36U0', '[ 30.28 -97.75]': '72254', '[ 30.29 -97.75]': '72254', '[ 30.24 -97.75]': '72254', '[ 29.44 -98.49]': '72253', '[ 27.81 -97.08]': '69029', '[ 27.7  -97.29]': '4MIKP', '[ 27.71 -97.32]': '4MIKP', '[ 27.79 -97.39]': '4M

,name,country,region,wmo,icao,latitude,longitude,elevation,timezone,hourly_start,hourly_end,daily_start,daily_end,monthly_start,monthly_end,distance
id,,,,,,,,,,,,,,,,
KTKI0,McKinney,US,TX,<NA>,KTKI,33.1779,-96.5905,179.0,America/Chicago,2006-01-01,2023-10-18,2006-01-01,2022-04-25,2006-01-01,2022-01-01,7402.455335
72254,Camp Mabry/Austin City Asos,US,TX,72254,KATT,30.3167,-97.7667,201.0,America/Chicago,2000-01-01,2023-10-18,1938-06-01,2023-10-10,1938-01-01,2022-01-01,7588.073798
72253,San Antonio International Airport,US,TX,72253,KSAT,29.5333,-98.4667,247.0,America/Chicago,1946-08-14,2023-10-18,1946-08-14,2023-10-11,1946-01-01,2022-01-01,10454.051693
KAPA0,Denver / Dove Valley,US,CO,<NA>,KAPA,39.5701,-104.8493,1794.0,America/Denver,2006-01-01,2023-10-18,2006-01-01,2022-04-24,2006-01-01,2022-01-01,12660.544227
72469,"Denver / Stapleton International, Co.",US,CO,72469,KDNR,39.7833,-104.8667,1626.0,America/Denver,1940-05-15,1995-03-01,1940-05-15,2023-09-19,1940-01-01,2021-01-01,14883.565539


In [124]:
# Retrieving Weather Data from Nearby Stations

# Create dictionary to hold dataframes of weather data
weather_df_dict = {}
for coord, station_id in run_loc_dict.items():
    weather_df_dict[station_id] = pd.DataFrame()

for idx, station in weather_stations.iterrows():
    # Daily Weather
    weather_df_dict[idx] = Daily(idx, start=datetime.combine(min_date, datetime.min.time()), end=datetime.combine(max_date, datetime.min.time())).fetch()

    # Hourly Weather
    #weather_df_dict[idx] = Hourly(idx, start=datetime.combine(min_date, datetime.min.time()), end=datetime.combine(max_date, datetime.min.time())).fetch()

print(weather_df_dict['72469'])

            tavg  tmin  tmax  prcp  snow   wdir  wspd  wpgt    pres  tsun
time                                                                     
2022-02-19   4.0  -2.2  15.6   0.0   0.0  207.0  12.1   NaN  1019.0   NaN
2022-02-20   6.7  -1.7  18.3   0.0   0.0  215.0  14.1   NaN  1007.9   NaN
2022-02-21   2.2 -15.6  12.2   0.0   0.0  112.0  20.4   NaN  1003.3   NaN
2022-02-22 -10.8 -18.9 -13.3   1.8  25.0   50.0  23.4   NaN  1021.3   NaN
2022-02-23 -11.0 -18.9 -12.2   1.3  76.0   67.0  19.5   NaN  1022.9   NaN
...          ...   ...   ...   ...   ...    ...   ...   ...     ...   ...
2023-10-18  14.5   9.9  20.5   0.0   NaN   49.0  12.3   NaN  1016.2   NaN
2023-10-19  15.2   8.3  23.0   0.0   NaN  128.0   8.3   NaN  1019.2   NaN
2023-10-20  15.9   8.4  25.0   0.0   NaN  148.0   7.3   NaN  1016.1   NaN
2023-10-21  15.2   8.5  22.8   0.0   NaN   88.0   8.6   NaN  1012.3   NaN
2023-10-22  13.9   8.3  20.8   0.0   NaN   72.0   9.4   NaN  1008.9   NaN

[611 rows x 10 columns]


In [138]:
# Add weather data to run_activities dataframe
run_activities["weather_station"] = None
run_activities["avg_temp"] = None
for idx, activity in run_activities.iterrows():
    dict_key = "[ " + str(activity['start_lat']) + " " + str(activity['start_lng']) + "]"
    if dict_key in run_loc_dict:
        weather_df = weather_df_dict[run_loc_dict[dict_key]]
        try:
            run_activities.at[idx,"weather_station"] = run_loc_dict[dict_key]
            run_activities.at[idx,"avg_temp"] = weather_df.loc[str(activity["start_date_local"])]["tavg"]
        except:
            # weather not found for date / station
            run_activities.at[idx,"weather_station"] = run_loc_dict[dict_key]


print(run_activities.isnull().sum())

run_activities
    

start_date_local         0
distance                 0
moving_time              0
total_elevation_gain     0
type                     0
average_speed            0
max_speed                0
average_cadence         16
average_heartrate       15
max_heartrate           15
elev_high                7
start_time               0
start_lat                7
start_lng                7
weather_station         13
avg_temp                14
dtype: int64


,start_date_local,distance,moving_time,total_elevation_gain,type,average_speed,max_speed,average_cadence,average_heartrate,max_heartrate,elev_high,start_time,start_lat,start_lng,weather_station,avg_temp
0,2023-10-22,35424.9,9692,231.7,Run,3.655,5.278,81.9,159.9,183.0,214.6,09:33:50,33.18,-96.67,72253,24.6
2,2023-10-21,9676.9,2659,65.7,Run,3.639,4.494,81.0,152.2,177.0,220.0,13:08:26,33.18,-96.67,72253,23.2
4,2023-10-20,16113.3,4482,114.5,Run,3.595,4.635,80.6,156.5,175.0,207.3,12:24:38,33.17,-96.67,KSSF0,24.0
6,2023-10-19,12903.1,3522,88.3,Run,3.664,4.930,81.3,153.5,173.0,209.4,11:12:57,33.17,-96.67,72253,24.8
7,2023-10-18,8072.6,2296,71.5,Run,3.516,4.221,82.4,148.1,165.0,207.2,11:58:41,33.18,-96.67,72253,24.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2022-02-23,9683.4,2978,79.0,Run,3.252,8.452,81.4,140.7,156.0,170.3,07:16:11,30.27,-97.63,74745,5.6
196,2022-02-22,11280.7,3428,90.4,Run,3.291,3.980,80.8,146.5,169.0,170.2,07:56:45,30.27,-97.63,74745,22.4
197,2022-02-20,1137.2,619,0.0,Run,1.837,3.512,63.7,107.0,141.0,2.4,23:04:05,29.96,-90.06,KVSH0,22.7
198,2022-02-20,6777.9,2142,0.0,Run,3.164,3.753,82.2,133.5,151.0,1.3,09:53:23,29.97,-90.08,KVSH0,22.8


In [ ]:
# Pre-processing
# Handle Null values

# Standardize numerical columns (total_elevation_gain, max_heart_rate

# Handle start_time ??